# LAB 5. Hypothesis testing. Green taxi case

Test varios hypothesis about ridership volume and trip characteristics for green taxi trips in 2018-2020. 

Does the ridership decline over time? Does it depend on the day of the week and time of the day? Does the distribution of the trip parameters (distance, time, speed) depend on the day of the week and borough?

In [1]:
#import the modules
import pandas as pd
import geopandas as gpd
import urllib.request
import os
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [18]:
traffic = pd.read_csv('hourlytra.csv')

traffic


,Plaza ID,Date,Hour,Direction,# Vehicles - E-ZPass,# Vehicles - VToll
0,21,12/02/2023,0,I,2650,573
1,21,12/02/2023,1,I,1794,407
2,21,12/02/2023,2,I,1309,366
3,21,12/02/2023,3,I,1238,290
4,21,12/02/2023,4,I,1679,390
...,...,...,...,...,...,...
2139700,11,01/01/2010,19,I,2675,1152
2139701,11,01/01/2010,20,I,2580,1086
2139702,11,01/01/2010,21,I,2302,987
2139703,11,01/01/2010,22,I,2170,910


In [21]:
print(traffic.dtypes)

Plaza ID                 int64
Date                    object
Hour                     int64
Direction               object
# Vehicles - E-ZPass     int64
# Vehicles - VToll       int64
dtype: object


In [16]:
tfd = traffic[['Date', '# Vehicles - E-ZPass']].groupby(by='Date', as_index=False).sum()
tfd

,Date,# Vehicles - E-ZPass
0,01/01/2010,316187
1,01/01/2011,349606
2,01/01/2012,402116
3,01/01/2013,376570
4,01/01/2014,387433
...,...,...
4982,12/31/2018,558930
4983,12/31/2019,593659
4984,12/31/2020,588085
4985,12/31/2021,530881


In [31]:
tfd['Date'] = pd.to_datetime(tfd['Date'], format='%m/%d/%Y')
tfd

,Date,# Vehicles - E-ZPass
0,2010-01-01,316187
1,2011-01-01,349606
2,2012-01-01,402116
3,2013-01-01,376570
4,2014-01-01,387433
...,...,...
4982,2018-12-31,558930
4983,2019-12-31,593659
4984,2020-12-31,588085
4985,2021-12-31,530881


In [34]:
tfds = tfd.sort_values(by='Date')
tfds

,Date,# Vehicles - E-ZPass
0,2010-01-01,316187
14,2010-01-02,380746
28,2010-01-03,359420
42,2010-01-04,494168
56,2010-01-05,518537
...,...,...
4561,2023-11-28,796503
4575,2023-11-29,797478
4589,2023-11-30,823897
4603,2023-12-01,819864


In [37]:

tfds['Year'] = tfds['Date'].dt.year
tfds2023 = tfds.loc[tfds['Year'].isin([2023])]
tfds2023

,Date,# Vehicles - E-ZPass,Year
13,2023-01-01,622263,2023
27,2023-01-02,588411,2023
41,2023-01-03,708088,2023
55,2023-01-04,728725,2023
69,2023-01-05,745644,2023
...,...,...,...
4561,2023-11-28,796503,2023
4575,2023-11-29,797478,2023
4589,2023-11-30,823897,2023
4603,2023-12-01,819864,2023


In [38]:
NO2 = pd.read_csv('NY_NO2_2023.csv')
SO2 = pd.read_csv('NY_SO2_2023.csv')
CO = pd.read_csv('NY_CO_2023.csv')
Ozone = pd.read_csv('NY_Ozone_2023.csv')
PM2d5 = pd.read_csv('NY_PM2.5_2023.csv')
PM10 = pd.read_csv('NY_PM10_2023.csv')








In [39]:
NO2

,Date,Source,Site ID,POC,Daily Max 1-hour NO2 Concentration,UNITS,DAILY_AQI_VALUE,Site Name,DAILY_OBS_COUNT,PERCENT_COMPLETE,AQS_PARAMETER_CODE,AQS_PARAMETER_DESC,CBSA_CODE,CBSA_NAME,STATE_CODE,STATE,COUNTY_CODE,COUNTY,SITE_LATITUDE,SITE_LONGITUDE
0,01/01/2023,AQS,340030010,1,30.7,ppb,28,Fort Lee Near Road,21,88.0,42602,Nitrogen dioxide (NO2),35620,"New York-Newark-Jersey City, NY-NJ-PA",34,New Jersey,3,Bergen,40.853550,-73.966180
1,01/02/2023,AQS,340030010,1,33.7,ppb,31,Fort Lee Near Road,24,100.0,42602,Nitrogen dioxide (NO2),35620,"New York-Newark-Jersey City, NY-NJ-PA",34,New Jersey,3,Bergen,40.853550,-73.966180
2,01/03/2023,AQS,340030010,1,68.4,ppb,66,Fort Lee Near Road,24,100.0,42602,Nitrogen dioxide (NO2),35620,"New York-Newark-Jersey City, NY-NJ-PA",34,New Jersey,3,Bergen,40.853550,-73.966180
3,01/04/2023,AQS,340030010,1,33.3,ppb,31,Fort Lee Near Road,24,100.0,42602,Nitrogen dioxide (NO2),35620,"New York-Newark-Jersey City, NY-NJ-PA",34,New Jersey,3,Bergen,40.853550,-73.966180
4,01/05/2023,AQS,340030010,1,21.7,ppb,20,Fort Lee Near Road,24,100.0,42602,Nitrogen dioxide (NO2),35620,"New York-Newark-Jersey City, NY-NJ-PA",34,New Jersey,3,Bergen,40.853550,-73.966180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2503,10/26/2023,AQS,361030044,1,19.4,ppb,18,Flax Pond,24,100.0,42602,Nitrogen dioxide (NO2),35620,"New York-Newark-Jersey City, NY-NJ-PA",36,New York,103,Suffolk,40.961017,-73.139046
2504,10/27/2023,AQS,361030044,1,29.8,ppb,27,Flax Pond,24,100.0,42602,Nitrogen dioxide (NO2),35620,"New York-Newark-Jersey City, NY-NJ-PA",36,New York,103,Suffolk,40.961017,-73.139046
2505,10/28/2023,AQS,361030044,1,17.1,ppb,16,Flax Pond,24,100.0,42602,Nitrogen dioxide (NO2),35620,"New York-Newark-Jersey City, NY-NJ-PA",36,New York,103,Suffolk,40.961017,-73.139046
2506,10/29/2023,AQS,361030044,1,2.1,ppb,2,Flax Pond,24,100.0,42602,Nitrogen dioxide (NO2),35620,"New York-Newark-Jersey City, NY-NJ-PA",36,New York,103,Suffolk,40.961017,-73.139046


In [45]:
NO2 = NO2.loc[NO2['COUNTY'].isin(['Queens','Kings','New York','Richmond','Bronx'])]
SO2 = SO2.loc[SO2['COUNTY'].isin(['Queens','Kings','New York','Richmond','Bronx'])]
CO = CO.loc[CO['COUNTY'].isin(['Queens','Kings','New York','Richmond','Bronx'])]
PM2d5 = PM2d5.loc[PM2d5['COUNTY'].isin(['Queens','Kings','New York','Richmond','Bronx'])]
PM10 = PM10.loc[PM10['COUNTY'].isin(['Queens','Kings','New York','Richmond','Bronx'])]
Ozone = Ozone.loc[Ozone['COUNTY'].isin(['Queens','Kings','New York','Richmond','Bronx'])]
SO2

,Date,Source,Site ID,POC,Daily Max 1-hour SO2 Concentration,UNITS,DAILY_AQI_VALUE,Site Name,DAILY_OBS_COUNT,PERCENT_COMPLETE,AQS_PARAMETER_CODE,AQS_PARAMETER_DESC,CBSA_CODE,CBSA_NAME,STATE_CODE,STATE,COUNTY_CODE,COUNTY,SITE_LATITUDE,SITE_LONGITUDE
888,01/01/2023,AQS,360050110,1,1.7,ppb,1,IS 52,24,100.0,42401,Sulfur dioxide,35620,"New York-Newark-Jersey City, NY-NJ-PA",36,New York,5,Bronx,40.81600,-73.90200
889,01/02/2023,AQS,360050110,1,1.4,ppb,1,IS 52,24,100.0,42401,Sulfur dioxide,35620,"New York-Newark-Jersey City, NY-NJ-PA",36,New York,5,Bronx,40.81600,-73.90200
890,01/03/2023,AQS,360050110,1,1.9,ppb,1,IS 52,24,100.0,42401,Sulfur dioxide,35620,"New York-Newark-Jersey City, NY-NJ-PA",36,New York,5,Bronx,40.81600,-73.90200
891,01/04/2023,AQS,360050110,1,1.0,ppb,1,IS 52,21,88.0,42401,Sulfur dioxide,35620,"New York-Newark-Jersey City, NY-NJ-PA",36,New York,5,Bronx,40.81600,-73.90200
892,01/05/2023,AQS,360050110,1,0.4,ppb,0,IS 52,24,100.0,42401,Sulfur dioxide,35620,"New York-Newark-Jersey City, NY-NJ-PA",36,New York,5,Bronx,40.81600,-73.90200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1974,10/27/2023,AQS,360810124,2,1.2,ppb,1,QUEENS COLLEGE 2,24,100.0,42401,Sulfur dioxide,35620,"New York-Newark-Jersey City, NY-NJ-PA",36,New York,81,Queens,40.73614,-73.82153
1975,10/28/2023,AQS,360810124,2,1.1,ppb,1,QUEENS COLLEGE 2,24,100.0,42401,Sulfur dioxide,35620,"New York-Newark-Jersey City, NY-NJ-PA",36,New York,81,Queens,40.73614,-73.82153
1976,10/29/2023,AQS,360810124,2,0.1,ppb,0,QUEENS COLLEGE 2,24,100.0,42401,Sulfur dioxide,35620,"New York-Newark-Jersey City, NY-NJ-PA",36,New York,81,Queens,40.73614,-73.82153
1977,10/30/2023,AQS,360810124,2,1.6,ppb,1,QUEENS COLLEGE 2,24,100.0,42401,Sulfur dioxide,35620,"New York-Newark-Jersey City, NY-NJ-PA",36,New York,81,Queens,40.73614,-73.82153


In [46]:
NO2m = NO2[['Date','DAILY_AQI_VALUE']].groupby(by='Date', as_index=False).max()
SO2m = SO2[['Date','DAILY_AQI_VALUE']].groupby(by='Date', as_index=False).max()
COm = CO[['Date','DAILY_AQI_VALUE']].groupby(by='Date', as_index=False).max()
PM2d5m = PM2d5[['Date','DAILY_AQI_VALUE']].groupby(by='Date', as_index=False).max()
PM10m = PM10[['Date','DAILY_AQI_VALUE']].groupby(by='Date', as_index=False).max()
Ozonem = Ozone[['Date','DAILY_AQI_VALUE']].groupby(by='Date', as_index=False).max()
NO2m

,Date,DAILY_AQI_VALUE
0,01/01/2023,6
1,01/02/2023,6
2,01/03/2023,9
3,01/04/2023,10
4,01/05/2023,6
...,...,...
300,10/28/2023,5
301,10/29/2023,5
302,10/30/2023,7
303,10/31/2023,5


In [51]:
NO2m['Date'] = pd.to_datetime(NO2m['Date'], format='%m/%d/%Y')
SO2m['Date'] = pd.to_datetime(SO2m['Date'], format='%m/%d/%Y')
COm['Date'] = pd.to_datetime(COm['Date'], format='%m/%d/%Y')
PM2d5m['Date'] = pd.to_datetime(PM2d5m['Date'], format='%m/%d/%Y')
PM10m['Date'] = pd.to_datetime(PM10m['Date'], format='%m/%d/%Y')
Ozonem['Date'] = pd.to_datetime(Ozonem['Date'], format='%m/%d/%Y')
Ozonem

,Date,DAILY_AQI_VALUE
0,2023-01-06,8
1,2023-01-12,12
2,2023-01-18,9
3,2023-01-24,7
4,2023-01-30,17
5,2023-02-05,17
6,2023-02-11,6
7,2023-02-17,11
8,2023-02-23,7
9,2023-03-01,10


In [53]:
NO2m.rename(columns={'DAILY_AQI_VALUE': 'NO2m'}, inplace=True)
SO2m.rename(columns={'DAILY_AQI_VALUE': 'SO2m'}, inplace=True)
COm.rename(columns={'DAILY_AQI_VALUE': 'COm'}, inplace=True)
PM2d5m.rename(columns={'DAILY_AQI_VALUE': 'PM2d5m'}, inplace=True)
PM10m.rename(columns={'DAILY_AQI_VALUE': 'PM10m'}, inplace=True)
Ozonem.rename(columns={'DAILY_AQI_VALUE': 'Ozonem'}, inplace=True)

In [54]:
merged_AQI = pd.merge(NO2m, SO2m, on='Date', how='outer')
merged_AQI = pd.merge(merged_AQI, COm, on='Date', how='outer')
merged_AQI = pd.merge(merged_AQI, PM2d5m, on='Date', how='outer')
merged_AQI = pd.merge(merged_AQI, PM10m, on='Date', how='outer')
merged_AQI = pd.merge(merged_AQI, Ozonem, on='Date', how='outer')
merged_AQI

,Date,NO2m,SO2m,COm,PM2d5m,PM10m,Ozonem
0,2023-01-01,36.0,1.0,6.0,45,NaN,27
1,2023-01-02,36.0,3.0,6.0,54,NaN,13
2,2023-01-03,40.0,1.0,9.0,75,NaN,4
3,2023-01-04,33.0,6.0,10.0,66,NaN,7
4,2023-01-05,18.0,0.0,6.0,56,NaN,12
...,...,...,...,...,...,...,...
337,2023-12-04,NaN,NaN,NaN,27,NaN,16
338,2023-12-05,NaN,NaN,NaN,25,NaN,24
339,2023-12-06,NaN,NaN,NaN,23,NaN,24
340,2023-12-07,NaN,NaN,NaN,45,NaN,22
